# Run local LLMs with Ollama and Langchain

- [Install Ollama locally](https://github.com/ollama/ollama)
- Test

`ollama pull llama3`

`ollama run llama3`

- Available models for native download - https://ollama.com/library
- I also used [LM Studio](https://lmstudio.ai/) to search for and download models from HuggingFace , including community optimized quantized models
- For Ollama download a gguf from huggingface, create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), then e.g. 

`ollama create mixtral -f Modelfile`

- You can [quantize and fine-tune your own](https://www.answer.ai/posts/2024-03-06-fsdp-qlora.html)
- This is run on M3 Macbook Air with 24GB memory. The 9b-parameter Gemma2 model performs well on these tests. The 27b-parameter model is probably pretty amazing, but the biggest one I can run on this MacBook is 3-bit K-quantization (S) - says not recommended - https://huggingface.co/bartowski/gemma-2-27b-it-GGUF
- They run fast enough on M3 as long as you stick to models LM Studio says 'Full GPU Offload Possible'


In [4]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta
import sys
import yaml

import sqlite3
import pickle

# needed because jupyter is already running an async event loop
import asyncio, aiohttp
import nest_asyncio
nest_asyncio.apply()

import pdb

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from bs4 import BeautifulSoup

from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain_core.tools import StructuredTool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

import wikipedia

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

try:
    module_name = 'webscrape'
    del sys.modules[module_name]
except:
    pass

import webscrape

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [5]:
page = 'Eli Lilly and Company'

def fn_get_wiki_page(page):
    """Given the name of existing wikipedia page, return the content of the page"""
    return wikipedia.page(title=page, auto_suggest=False).content

page_content = fn_get_wiki_page(page)
print(len(page_content))
page_content[:1000]


56222


'Eli Lilly and Company is an American pharmaceutical company headquartered in Indianapolis, Indiana, with offices in 18 countries. Its products are sold in approximately 125 countries. The company was founded in 1876 by Eli Lilly, a pharmaceutical chemist and Union Army veteran of the American Civil War for whom the company was later named.\nAs of 2024, the company is ranked 127th on the Fortune 500 with revenue of $34.12 billion. It is ranked 221st on the Forbes Global 2000 list of the world\'s largest publicly-traded companies and 252nd on the Forbes list of "America\'s Best Employers". It is recognized as the top entry-level employer in Indianapolis.\nLilly is known for its clinical depression drugs Prozac (fluoxetine) (1986), Cymbalta (duloxetine) (2004), and its antipsychotic medication Zyprexa (olanzapine) (1996). The company\'s primary revenue drivers are the diabetes drugs Humalog (insulin lispro) (1996) and Trulicity (dulaglutide) (2014).\nLilly was the first company to mass-p

In [32]:
prompts = [
"Translate ‘You are amazing and I love you’ into French.",
"Define ‘photosynthesis’.",
"Copy edit this sentence for grammar, spelling, and punctuation: ‘She decide too bake cookies for her friends but then she realized she didn’t have enough suger.’",
"Write a humorous limerick about hedge funds where the first letter of each verse spells 'hedge'.",
f"""You will act as a securities analyst with deep knowledge of financial markets,
securities analysis, and portfolio management. You will maintain a professional yet engaging tone,
in the style of a senior investment bank research analyst. You will concisely provide facts
relevant to investors and avoid opinions unless specifically requested.

You will focus on Eli Lilly (LLY). You will summarize the provided content below from
Wikipedia for Eli Lilly and summarize all topics of interest to investors on this page in 3000
words or less: 
{page_content}""",
]


In [7]:
models = [
#     "Yi_lm",
#     "gemma_lm",
#     "yi",
#     "aya",
    "gemma2",
    "phi3",
    "qwen2",
#     "llama3",
#     "mistral",
#     "mixtral",
]


In [8]:
local_llm = ChatOllama(model=models[0], temperature=0)

messages = [
    SystemMessage(content="You are a friendly assistant."),
    HumanMessage(content=prompts[0]),
]

response = local_llm.invoke(messages)

print(response.content, "\n")


"Tu es incroyable et je t'aime." 


Let me know if you'd like me to translate anything else! 😊  
 



In [9]:
system_template = "You are a helpful assistant."

all_responses = ""

for m in models:
    
    print(f"# {m}\n")
    response = f"# {m}\n"
    local_llm = ChatOllama(model=m, temperature=0)
    
    for p in prompts:
        print(p[:200])

        prompt_template = ChatPromptTemplate.from_messages(
            [("system", system_template),
             ("user", p)]
        )

        parser = StrOutputParser()

        chain = prompt_template | local_llm | parser

        # stream tokens as they are generated
        for r in chain.stream({}):
            print(r, end="")
            response += r

        print("\n\n---\n\n")
        response += "\n\n---\n\n"
        
        
    all_responses += response
    all_responses += "\n\n---\n\n"
        
        
# display(Markdown(all_responses))

# response


# gemma2

Translate ‘You are amazing and I love you’ into French.
"Tu es incroyable et je t'aime." 


Let me know if you'd like me to translate anything else! 😊  


---


Define ‘photosynthesis’.
Photosynthesis is the amazing process used by plants, algae, and some bacteria to convert light energy from the sun into chemical energy in the form of sugars. 

Here's a simple breakdown:

* **Sunlight:** Plants capture light energy using a green pigment called chlorophyll.
* **Water:** Plants absorb water from the soil through their roots.
* **Carbon Dioxide:** Plants take in carbon dioxide from the air through tiny pores on their leaves called stomata.

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the spec

"Vous êtes incroyable et je vous aime"

---


Define ‘photosynthesis’.
Photosynthesis is the process used by plants, algae, and certain bacteria to convert light energy into chemical energy that can be used to fuel their growth. This process occurs in the chloroplasts of plant cells, which contain a green pigment called chlorophyll.

During photosynthesis, these organisms absorb carbon dioxide (CO2) from the air and water (H2O) from the soil using sunlight as an energy source. The light energy is captured by chlorophyll and used to convert CO2 and H2O into glucose (C6H12O6), a simple sugar that serves as food for the plant, and oxygen (O2), which is released into the air.

The overall chemical equation for photosynthesis can be represented as:

\[ 6CO_2 + 6H_2O + light \rightarrow C_6H_{12}O_6 + 6O_2 \]

This process not only provides energy for plants but also plays a crucial role in the global carbon cycle and is responsible for producing most of the oxygen found in Earth's atmospher

In [10]:
display(Markdown(all_responses))


# gemma2
"Tu es incroyable et je t'aime." 


Let me know if you'd like me to translate anything else! 😊  


---

Photosynthesis is the amazing process used by plants, algae, and some bacteria to convert light energy from the sun into chemical energy in the form of sugars. 

Here's a simple breakdown:

* **Sunlight:** Plants capture light energy using a green pigment called chlorophyll.
* **Water:** Plants absorb water from the soil through their roots.
* **Carbon Dioxide:** Plants take in carbon dioxide from the air through tiny pores on their leaves called stomata.

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the specific steps involved in photosynthesis or its importance to life on Earth.

---

She decided to bake cookies for her friends, but then she realized she didn't have enough sugar. 


---

**H**igh-rolling traders, sharp and keen,
**E**ager to make fortunes unseen.
**D**ealing in stocks and more,
**G**etting rich, that's for sure,
**E**ven if the market's a mean machine. 


Let me know if you'd like to try another limerick! 


---

This text provides a comprehensive overview of Eli Lilly and Company, covering its history, community service, controversies, and more. 

Here's a breakdown of the key points:

**History:**

* Founded in 1876 by Colonel Eli Lilly in Indianapolis, Indiana.
* Initially focused on producing morphine and other pharmaceutical products.
* Expanded its product line to include insulin, Prozac, Zyprexa, and other medications.
* Acquired by its current parent company in 1987.

**Community Service:**

* Long history of philanthropy and support for local organizations.
* Provided aid to disaster victims, including the San Francisco earthquake and the Johnstown Flood.
* Established Lilly Field Hospital 32 during World War I.
* Manufactured over 200 products for military use during World War II.

**Foundations:**

* **Lilly Endowment:** Founded in 1937 by Josiah K. Lilly Sr. and his sons, Eli and Joe.
* **Eli Lilly and Company Foundation:** Established in 1968, funded through corporate profits.

**Controversies:**

* **BGH:** Purchase of bovine growth hormone (BGH) from Monsanto in 2008, facing ethical and health concerns.
* **340B:** Legal battle with the Department of Health and Human Services over drug pricing for hospitals.
* **Prozac:** Lawsuits and public outcry following a 1989 mass shooting linked to the drug.
* **Zyprexa:** Multiple lawsuits and criminal charges for illegally marketing the drug for unapproved uses.
* **Discrimination:** Accused of sex discrimination and age discrimination in hiring practices.
* **Canada Patent Lawsuit:** Sued Canada for violating its obligations to foreign investors.
* **Evista:** Guilty plea and fine for illegally promoting the drug for unapproved uses.
* **Insulin Pricing:** Facing criticism for high insulin prices and slow response to calls for affordability.

**Overall:**

The text paints a complex picture of Eli Lilly and Company, highlighting its contributions to medicine and society alongside its involvement in controversies and legal battles. It demonstrates the challenges faced by pharmaceutical companies in balancing innovation, profitability, and ethical considerations.




---



---

# phi3
 Vous êtes incroyable et j'aime vous.

---

 Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose or other sugars. This complex biochemical reaction occurs primarily within specialized cell structures called chloroplasts found in plant cells.

During photosynthesis, plants use carbon dioxide (CO2) and water (H2O), along with sunlight energy, to produce glucose (C6H12O6) and oxygen gas (O2). The overall chemical equation for this process can be represented as:

6 CO2 + 6 H2O + light energy → C6H12O6 + 6 O2

Photosynthesis consists of two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle). In the first stage, sunlight is absorbed by chlorophyll pigments in the thylakoid membranes within the chloroplasts. This energy is used to split water molecules into hydrogen ions and oxygen gas, while also generating ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate), which are essential for the second stage of photosynthesis.

In the light-independent reactions or Calvin cycle, the energy stored in ATP and NADPH is used to convert carbon dioxide into glucose through a series of enzyme-catalyzed chemical reactions. The produced glucose can then be utilized by plants for growth, reproduction, and other metabolic processes or stored as starch for later use.

Photosynthesis is essential to life on Earth, as it provides the primary source of energy for most living organisms through food chains and contributes significantly to oxygen production in our atmosphere.

---

 She decided to bake cookies for her friends, but then she realized she didn't have enough sugar.

(Note: "decide" was corrected to its base form "decided," and "too" was changed to "to.")

---

 Hedge funds, oh how they grow,
Eager investors in tow.
Dollars dance and sway,
Gains come out on display,
Everyone's wealth just starts to flow!

---

 Eli Lilly and Company, founded in 1876 by Colonel John E. Lilly, is a global biopharmaceecial company headquartered in Indianapolis, Indiana. It has grown to become one of the largest pharmaceutical companies worldwide, with its products spanning across various therapeutic areas including oncology, diabetes care, and neuroscience.

The company's history is marked by significant milestones such as the development of insulin in 1923, which revolutionized diabetes treatment. Eli Lilly also faced legal challenges over marketing practices, including an illegal promotion of Evista for conditions not approved by the FDA and a controversy surrounding rising insulin prices.

Eli Lilly's commitment to innovation is evident in its extensive research and development efforts, with numerous patents granted annually. The company has also been involved in various philanthropic activities, including supporting education through scholarships and grants.

Despite facing criticism for high drug prices, Eli Lilly continues to invest heavily in R&D, aiming to address unmet medical needs and improve patient outcomes. Its recent initiatives include reducing the cost of insulin and offering discounts on certain medications, reflecting a response to public concerns over healthcare affordability.

Eli Lilly's impact extends beyond its products; it has contributed significantly to Indiana's economy and cultural heritage, with historical landmarks like Building 314 in Indianapolis serving as reminders of the company's legacy.

---



---

# qwen2
"Vous êtes incroyable et je vous aime"

---

Photosynthesis is the process used by plants, algae, and certain bacteria to convert light energy into chemical energy that can be used to fuel their growth. This process occurs in the chloroplasts of plant cells, which contain a green pigment called chlorophyll.

During photosynthesis, these organisms absorb carbon dioxide (CO2) from the air and water (H2O) from the soil using sunlight as an energy source. The light energy is captured by chlorophyll and used to convert CO2 and H2O into glucose (C6H12O6), a simple sugar that serves as food for the plant, and oxygen (O2), which is released into the air.

The overall chemical equation for photosynthesis can be represented as:

\[ 6CO_2 + 6H_2O + light \rightarrow C_6H_{12}O_6 + 6O_2 \]

This process not only provides energy for plants but also plays a crucial role in the global carbon cycle and is responsible for producing most of the oxygen found in Earth's atmosphere.

---

"She decided to bake cookies for her friends, but then she realized she didn't have enough sugar."

---

There once was a fund quite bold,
Whose name started with "H" and gold,
In markets it roared,
With strategies galore,
But always kept folks on the hold.  

Its managers, so sly,
Would bet on what's up or low,
With leverage they'd play,
And portfolios they'd sway,
Yet, risks were not to be told.

In tales of its trade,
Some saw wealth in their stride,
While others would dread,
The volatility spread,
For "hedge" was a code for surprise.  

So here ends our rhyme,
Of funds that might climb or decline,
Remember the rule:
When investing, beware the fool,
And hedge your bets with care and line.

---

Eli Lilly and Company is a global pharmaceutical company founded in Indianapolis, Indiana by Colonel Eli Lilly in 1876. The company's history includes significant contributions to the field of medicine through research and development of various drugs.

Key points about Eli Lilly:

- **Innovation**: Eli Lilly has been at the forefront of innovation with numerous breakthroughs including insulin production for diabetes management, antipsychotic medication Zyprexa, and cancer treatments like Evista. 

- **Legal Issues**: The company faced significant legal challenges over marketing practices, particularly concerning its drug Zyprexa which led to large settlements due to allegations of illegal promotion.

- **Patent Disputes**: Eli Lilly has also been involved in patent disputes including a lawsuit against Canada for alleged violations under the North American Free Trade Agreement regarding patents on drugs Strattera and Zyprexa.

- **Insulin Pricing Controversy**: In recent years, Eli Lilly faced criticism over insulin pricing, with accusations of price gouging leading to investigations by US lawmakers. 

- **Corporate Responsibility**: Despite these challenges, Eli Lilly has been recognized for its contributions to healthcare and has implemented measures to ensure integrity in business practices.

Eli Lilly continues to operate globally, contributing significantly to the pharmaceutical industry through research and development while facing ongoing scrutiny over its operations and practices.

---



---



In [15]:
SQLITE_DB = "/Users/drucev/projects/AInewsbot/articles.db"

datestr = '2024-06-30'

conn = sqlite3.connect(SQLITE_DB)

c = conn.cursor()
query = f"select * from news_articles where article_date > '{datestr}' and isAI=1 order by id desc limit 171"
df = pd.read_sql_query(query, conn)
df


,id,src,title,url,isAI,article_date
0,118384,VentureBeat,Is Groq the David to Nvidia's Goliath? Groq CE...,https://venturebeat.com/ai/is-groq-the-david-t...,1,2024-07-01
1,118383,VentureBeat,Runway's Gen-3 Alpha AI video model now availa...,https://venturebeat.com/ai/runways-gen-3-alpha...,1,2024-07-01
2,118382,The Verge,Instagram’s ‘Made with AI’ label swapped out f...,https://www.theverge.com/2024/7/1/24190026/met...,1,2024-07-01
3,118379,The Register,Lambda on the hunt for 'another $800M' to fuel...,https://www.theregister.com/2024/07/01/lambda_...,1,2024-07-01
4,118377,The Register,France poised to bring 'charges against Nvidia...,https://www.theregister.com/2024/07/01/french_...,1,2024-07-01
...,...,...,...,...,...,...
166,117962,Feedly AI,Nvidia rival Groq AI set to raise $300M in fun...,https://nypost.com/2024/07/01/business/nvidia-...,1,2024-07-01
167,117961,Feedly AI,Blackmagic Adds “No AI Training” Section to Cl...,https://nofilmschool.com/blackmagic-ai-trainin...,1,2024-07-01
168,117952,Business Insider,AI startup Abnormal Security is set to be valu...,https://www.businessinsider.com/abnormal-secur...,1,2024-07-01
169,117951,Business Insider,Elon Musk reveals how many Nvidia H100 chips h...,https://www.businessinsider.com/elon-musk-hint...,1,2024-07-01


In [16]:
print(df.iloc[143].url)


https://www.futurity.org/artificial-intelligence-power-outages-3235522/


In [17]:
# Fetch HTML files from sources
# empty download directory
DOWNLOAD_DIR = webscrape.DOWNLOAD_DIR

# Iterate through the files in the directory and delete them
for filename in os.listdir(DOWNLOAD_DIR):
    file_path = os.path.join(DOWNLOAD_DIR, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print(f"All files in {DOWNLOAD_DIR} have been deleted.")

# save each file specified from sources
print("Saving HTML files")


All files in htmldata have been deleted.
Saving HTML files


In [19]:
# Create a queue for multiprocessing and populate it 
print("Enqueuing URLs")

queue = multiprocessing.Queue()
for row in df.itertuples():
    queue.put((row.url, row.title))

Enqueuing URLs


In [ ]:
# webscrape.get_url('https://www.theregister.com/2024/07/01/database_ai_training/')
# 

In [20]:
# process queue asynchronously

# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
def process_queue_factory(q):
    def process_queue():
        # launch browser via selenium driver
        driver = webscrape.get_driver()    
        saved_pages = []
        while not q.empty():
            url, title = q.get()
            webscrape.log(f'Processing {url}')
            savefile = webscrape.get_url(url, driver)
            saved_pages.append((url, title, savefile))
        # Close the browser
        webscrape.log("Quit webdriver")
        driver.quit()
        return saved_pages
    return process_queue

callable = process_queue_factory(queue)
# saved_pages = callable()

def launch_drivers(num_drivers):
    with ThreadPoolExecutor(max_workers=num_drivers) as executor:
        # Create a list of future objects
        futures = [executor.submit(callable) for _ in range(num_drivers)]
        
        # Collect the results (web drivers) as they complete
        retarray = [future.result() for future in as_completed(futures)]
        
    return retarray

results = launch_drivers(4)


In [21]:
# flatten results
saved_pages = []
for r in results:
    saved_pages.extend(r)
saved_pages


[('https://www.theregister.com/2024/07/01/french_nvidia_competition/',
  "France poised to bring 'charges against Nvidia'Euro nation's monopoly gendarmes cheesed off with GPU giant's dominanceAI + ML37 mins|",
  'htmldata/France_poised_to_bring__charges_against_Nvidia____The_Register_20240701_211412.html'),
 ('https://www.reddit.com/r/technology/comments/1dsvefb/googles_ai_search_summaries_use_10x_more_energy/',
  "Google's AI search summaries use 10x more energy than just doing a normal Google search",
  'htmldata/Google_s_AI_search_summaries_use_10x_more_energy_than_just_doing_a_normal_Google_search___r_technology_20240701_211424.html'),
 ('https://hackernoon.com/from-zero-to-ai-image-analyzer-in-5-minutes-a-beginners-guide',
  "From Zero to AI Image Analyzer in 5 Minutes: A Beginner's Guide",
  'htmldata/From_Zero_to_AI_Image_Analyzer_in_5_Minutes__A_Beginner_s_Guide___HackerNoon_20240701_211435.html'),
 ('https://hackernoon.com/life-in-2050-according-to-gemini-15-pro',
  'Life in 2

In [22]:
len(saved_pages)

171

In [24]:
# Open the file in write-binary mode and save the list of tuples
with open('saved_pages.pkl', 'wb') as file:
    pickle.dump(saved_pages, file)

print(f"saved_pages saved to {filename}")

List of tuples saved to Meet_CriticGPT__OpenAI_s_New_Fix_for_ChatGPT_s_Biggest_Coding_Blunders_-_Business_Insider_20240701_124656.html


In [25]:
print(datetime.now())


2024-07-01 21:22:29.752795


In [26]:
# summarize pages
SYSTEM_PROMPT = """You are a summarization assistant. 
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will output Markdown format.
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.
"""


In [ ]:
local_llm = ChatOllama(model='phi3', temperature=0)

all_responses = ""

for i, (url, title, filename) in enumerate(saved_pages):
    # Read the HTML file
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            html_content = file.read()
    except:
        print(f"Skipping {filename}")
        continue

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Filter out script and style elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.extract()

    # Get text and strip leading/trailing whitespace
    visible_text = soup.get_text(separator=' ', strip=True)

    userprompt = """Summarize the main points of the following text concisely in at most 3 bullet points:
{text}
"""
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", SYSTEM_PROMPT),
         ("user", userprompt)]
    )
    parser = StrOutputParser()

    chain = prompt_template | local_llm | parser

    response = chain.invoke({'text': visible_text})
    
    output = f"[{i+1}{title}]({url})\n\n"
    output += response
    output += "-" * 10
    output += "\n\n"
    display(Markdown(output))

    all_responses += output

In [27]:
print(datetime.now())

2024-07-01 21:22:38.842619


In [28]:
# running GPT 3.5 async with many parallel requests, you get a huge speedup vs. a local model
# cost is 1/10 the cost of GPT-4o
# the local models are still pretty slow and inconsistent
# Gemma-2-27B-it ranks above older GPT-4 models in LMSYS ranking and way above this model though

API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}

async def fetch_openai(session, payload):
    """
    Asynchronously fetches a response from the OpenAI URL using an aiohttp ClientSession.

    Parameters:
    - session (aiohttp.ClientSession): The aiohttp ClientSession object used for making HTTP requests.
    - payload (dict): The payload to be sent in the request body as JSON.

    Returns:
    - dict: The full JSON response from the OpenAI API.

    Raises:
    - aiohttp.ClientError: If there is an error during the HTTP request.

    Example usage:
    ```
    async with aiohttp.ClientSession() as session:
        response = await fetch_openai(session, payload)
        print(response)
    ```
    """
    async with session.post(API_URL, headers=headers, json=payload) as response:
        return await response.json()
    

In [36]:
payload = {"model":  "gpt-3.5-turbo-0125",
            "messages": [{"role": "system",
                          "content": SYSTEM_PROMPT
                         },
                         {"role": "user",
                          "content": prompts[4]
                         }]
          }
           

async with aiohttp.ClientSession() as session:            
    response = asyncio.run(fetch_openai(session, payload))

In [38]:
response_str = response["choices"][0]["message"]["content"]
print(response_str)

- Founded in 1876 by Eli Lilly, Eli Lilly and Company is an American pharmaceutical firm headquartered in Indianapolis, with operations in 18 countries and products sold in about 125 countries.
- The company is known for clinical depression drugs like Prozac, Cymbalta, and antipsychotic medication Zyprexa, with primary revenue drivers being diabetes drugs like Humalog and Trulicity.
- Famous for being the first to mass-produce the polio vaccine and insulin, Eli Lilly has faced legal issues such as a $1.415 billion penalty for illegally marketing Zyprexa and has been involved in various charitable initiatives and community service activities.


In [42]:
async def myfunc():
    tasks = []
    responses = []
    async with aiohttp.ClientSession() as session:

        for i, (url, title, filename) in enumerate(saved_pages):

            # Read the HTML file
            try:
                with open(filename, 'r', encoding='utf-8') as file:
                    html_content = file.read()
            except:
                print(f"Skipping {filename}")
                continue

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')

            # Filter out script and style elements
            for script_or_style in soup(['script', 'style']):
                script_or_style.extract()

            # Get text and strip leading/trailing whitespace
            visible_text = soup.get_text(separator=' ', strip=True)

            userprompt = f"""Summarize the main points of the following text concisely in at most 3 bullet points:
            {visible_text}
            """

            payload = {"model":  "gpt-3.5-turbo-0125",
                        "messages": [{"role": "system",
                                      "content": SYSTEM_PROMPT
                                     },
                                     {"role": "user",
                                      "content": userprompt
                                     }]
                       }
            
            task = asyncio.create_task(fetch_openai(session, payload, i))
            tasks.append(task)
            
        responses = await asyncio.gather(*tasks)
    return responses



In [45]:
responses = await myfunc()
print(len(responses))
responses[0]

171


{'id': 'chatcmpl-9gMk6f8olLeUJOVMgJpbl6Xl11Mm0',
 'object': 'chat.completion',
 'created': 1719884226,
 'model': 'gpt-3.5-turbo-0125',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "- French regulators are preparing to bring charges against Nvidia over alleged anti-competitive practices in the AI infrastructure market.\n- Concerns have been raised about Nvidia's dominant market position, including issues such as price fixing, production restrictions, and dependencies on Nvidia's CUDA chip programming software.\n- The European Union and the US Department of Justice are also looking into Nvidia's market dominance to determine if corrective action is necessary."},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1651,
  'completion_tokens': 81,
  'total_tokens': 1732},
 'system_fingerprint': None}

In [48]:
response_str_list = []
for response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        display(Markdown(response_str))
        display(Markdown("--" * 40))
        response_str_list.append(response_str)

    except Exception as exc:
        print(exc)
        
        
len(response_str_list)



- French regulators are preparing to bring charges against Nvidia over alleged anti-competitive practices in the AI infrastructure market.
- Concerns have been raised about Nvidia's dominant market position, including issues such as price fixing, production restrictions, and dependencies on Nvidia's CUDA chip programming software.
- The European Union and the US Department of Justice are also looking into Nvidia's market dominance to determine if corrective action is necessary.

--------------------------------------------------------------------------------

- Google's AI search summaries consume 10 times more energy than a regular Google search
- Users criticize the AI summaries for being misleading and unhelpful, sometimes suggesting unsafe actions
- Concerns raised about the inefficiency of AI technology compared to traditional database searches

--------------------------------------------------------------------------------

- Building an AI image analyzer using Project IDX and Gemini API without needing to set up anything.
- Steps involve opening Project IDX, choosing a template, obtaining an API key, and completing the image upload functionality.
- Testing the application to see Gemini's analysis of the uploaded image, showing it identifies objects and suggests related information.

...

- Groq AI, a startup competing with Nvidia in the AI chip market, is set to raise $300 million in funding led by BlackRock and other investors.
- Groq, founded by ex-Google executive Jonathan Ross, specializes in chips and software for generative AI products and aims to challenge Nvidia with faster and cheaper hardware.
- The startup has raised $367 million prior to this round and has faced controversies, including a firing incident with early backer Social Capital, led by Chamath Palihapitiya.

--------------------------------------------------------------------------------

- AI is utilized to develop a one-size-fits-all exoskeleton that adjusts to different users without individual adjustments, making it more feasible and user-friendly.
- The exoskeleton is created using AI-generated control policies, reducing the need for costly handcrafting and individual human training, potentially lowering prices to $2,000 to $5,000.
- Neural networks are trained to simulate human-exoskeleton interactions, resulting in record-breaking metabolic rate reductions during walking, running, and stair climbing with the exoskeleton.

--------------------------------------------------------------------------------

169

In [ ]:
    try:
        response_str = r["choices"][0]["message"]["content"]
    except Exception as exc:
        print(exc)
        return ""

    output = f"[{i+1}. {title}]({url})\n\n"
    output += response
    output += "-" * 10
    output += "\n\n"
    display(Markdown(output))

    all_responses += output

all_responses = ""

In [ ]:
response



In [ ]:
# classify relevance using local model
# add scraped title to database via an update
# consider the issue of reddit where the title may be reused 
# filter saved pages for html files that already exists using title